##PIPS

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


In [ ]:
from transformers import BertTokenizer, AutoTokenizer

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

##function definitions

In [ ]:
def calc_len(sent):
  word_count = len(sent.split())
  return word_count

    

In [ ]:
def calc_len_wtokenizer(sent):
  input_ids = tokenizer_bert.encode(sent, add_special_tokens=True)
  length = len(input_ids)
  return length

In [ ]:
def making_chunks(test_sent):
  tok_len = calc_len_wtokenizer(test_sent)

  n = int(np.ceil(tok_len/450))
  len_each_chunk = int(np.floor(450/n))

  word_count = len(test_sent.split())
  slice_var = int(word_count/n)

  list_sent = test_sent.split()
  chunk_sents = []
  
  start = 0
  last = slice_var
  for i in range(n):
    x = list_sent[start:last]
    res_str = "default: "

    res_str = ' '.join(x)
    #print(res_str)
    chunk_sents.append(res_str)
    start = last
    last = last+slice_var

  #print(n)
  return chunk_sents,n

In [ ]:
def mT5_summarize(str_w,n):
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
  len_each_chunk = int(np.floor(450/n))

  article_text = str_w

  input_ids = tokenizer(
      [WHITESPACE_HANDLER(article_text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]

  output_ids = model.generate(
      input_ids=input_ids.cuda(),
      max_length=len_each_chunk,
      min_length=len_each_chunk-100,
      no_repeat_ngram_size=2,
      num_beams=4
  )[0]

  summary = tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )

  #print(summary)
  return summary
    

## Load dataset
you only need to edit this part of the code from now on

In [ ]:
path1 = '/content/drive/MyDrive/datasets/aug_train_augx2_cbuet_para.csv' #initial iteration load
path2 = '/content/drive/MyDrive/datasets/sum_aug_train_augx2_cbuet_para.csv' #where it will be stored. and loaded for all next iterations

In [ ]:
#for initial iteration run this codes

# split1 = pd.read_csv(path1)
# split1.to_csv(path2, index=False)

In [ ]:
split1 = pd.read_csv(path1)

##Code for Summarization

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



In [ ]:
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [ ]:
# from transformers import BertTokenizer, AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer_bert = AutoTokenizer.from_pretrained('Tahsin-Mayeesha/bangla-fake-news-mbert')

Loading BERT tokenizer...


##example

In [ ]:
count_s=0
for sent in split1['text']:
  l = calc_len(sent)
  if l > 250 and l<270:
    break
   
print(calc_len(sent))
print(sent)


264
কোকাকোলার কোমল পানীয়তে মিশ্রিত হতে যাচ্ছে গাঁজা!। কোকা-কোলা এবার পানীয়তে গাঁজার নির্যাস যোগ করতে যাচ্ছে। কোকা-কোলা বলছে, গাঁজা সংশ্লিষ্ট পানীয়ের বাজার পর্যবেক্ষণ করছে তারা।কানাডাভিত্তিক বিএনএন ব্লুমবার্গ টিভি এ তথ্য জানিয়েছে। এছাড়াও স্কাইনিউজ, ম্যাশেবল, এবিসি নিউজ সহ বিশ্বের প্রথম সারির গণমাধ্যম এ খবর প্রকাশ করেছে।বলা হচ্ছে, স্থানীয় উৎপাদক ‘অরোরা ক্যানাবিস’ এর সঙ্গে গাঁজার স্বাদযুক্ত কোমল পানীয় উৎপাদনের বিষয়ে আলোচনা করছে কোকা-কোলা। তবে গ্রাহকদের মাদকাসক্ত করতে নয়, তাদের শারীরিক যন্ত্রণা লাঘব করাই পানীয় তৈরিকারীদের লক্ষ্য।কোকা-কোলা এক বিবৃতিতে জানিয়েছে, অনেক উৎপাদকের মতো আমরাও পর্যবেক্ষণ করছি যে, কোমল পানীয় তৈরির ক্ষেত্রে নন-সাইকোঅ্যাক্টিভ ক্যানাবিডিওল বা চিত্ত উত্তেজিত করে না এমন গাঁজা জাতীয় দ্রব্যের ব্যবহার কতটা জনপ্রিয়তা পাচ্ছে। ক্যানাবিডিওল ক্যানাবিস বা গাঁজার একটি উপাদান, যা প্রদাহ, ব্যথা বা খিঁচুনির চিকিৎসার ক্ষেত্রে আরামদায়ক হতে পারে এবং এর কোনো চিত্ত উত্তেজক প্রভাব নেই।গাঁজার বিনোদনমূলক ব্যবহার আইনত বৈধ করতে যাচ্ছে কানাডা। চিকিৎসা কাজে অবশ্য অনেক আগে থেকেই গাঁ

In [ ]:
chunks,n = making_chunks(sent)
sum_sent=""

for i in range(n):
  chunks[i] = mT5_summarize(chunks[i],n)

sum_sent = ' '.join(chunks)

In [ ]:
sent

'কোকাকোলার কোমল পানীয়তে মিশ্রিত হতে যাচ্ছে গাঁজা!। কোকা-কোলা এবার পানীয়তে গাঁজার নির্যাস যোগ করতে যাচ্ছে। কোকা-কোলা বলছে, গাঁজা সংশ্লিষ্ট পানীয়ের বাজার পর্যবেক্ষণ করছে তারা।কানাডাভিত্তিক বিএনএন ব্লুমবার্গ টিভি এ তথ্য জানিয়েছে। এছাড়াও স্কাইনিউজ, ম্যাশেবল, এবিসি নিউজ সহ বিশ্বের প্রথম সারির গণমাধ্যম এ খবর প্রকাশ করেছে।বলা হচ্ছে, স্থানীয় উৎপাদক ‘অরোরা ক্যানাবিস’ এর সঙ্গে গাঁজার স্বাদযুক্ত কোমল পানীয় উৎপাদনের বিষয়ে আলোচনা করছে কোকা-কোলা। তবে গ্রাহকদের মাদকাসক্ত করতে নয়, তাদের শারীরিক যন্ত্রণা লাঘব করাই পানীয় তৈরিকারীদের লক্ষ্য।কোকা-কোলা এক বিবৃতিতে জানিয়েছে, অনেক উৎপাদকের মতো আমরাও পর্যবেক্ষণ করছি যে, কোমল পানীয় তৈরির ক্ষেত্রে নন-সাইকোঅ্যাক্টিভ ক্যানাবিডিওল বা চিত্ত উত্তেজিত করে না এমন গাঁজা জাতীয় দ্রব্যের ব্যবহার কতটা জনপ্রিয়তা পাচ্ছে। ক্যানাবিডিওল ক্যানাবিস বা গাঁজার একটি উপাদান, যা প্রদাহ, ব্যথা বা খিঁচুনির চিকিৎসার ক্ষেত্রে আরামদায়ক হতে পারে এবং এর কোনো চিত্ত উত্তেজক প্রভাব নেই।গাঁজার বিনোদনমূলক ব্যবহার আইনত বৈধ করতে যাচ্ছে কানাডা। চিকিৎসা কাজে অবশ্য অনেক আগে থেকেই গাঁজা 

In [ ]:
sum_sent

'কোকাকোলার কোমল পানীয়তে মিশ্রিত হতে যাচ্ছে গাঁজা! এমন খবর প্রকাশ করেছে বিশ্বের প্রথম সারির গণমাধ্যম এবং সামাজিক যোগাযোগ মাধ্যমের বেশ কয়েকজন বিএনএন নিউজ স্ট্রিমিং এর সহযোগী সংগঠন ‘কোকা-কোলা’ ।  এখবর পাওয়া গেছে, যুক্তরাষ্ট্রের একটি উৎপাদকের সঙ্গে আলোচনা করার পর তারা বলছে যে, " এই  করোনাভাইরাসের সংক্রমণ ঠেকাতে যুক্তরাষ্ট্রের গাঁজা উৎপাদনকারী সংস্থা কোকা-কোলা আর ক্যানাডায় নতুন একটি নন-অ্যালকোহলিক পানীয় তৈরি করার বিষয়ে অরোরা’র অংশীদারিত্ব পাওয়ার সিদ্ধান্ত নিয়েছে দেশটির স্বাস্থ্য মন্ত্রণালয়ের সূত্র। কিন্তু এই চুক্তিতে কোক’এর সঙ্গে আলোচনা অগ্রসর হওয়ার পর তারা বলছে, এখন থেকে  .. ।'

###chunks

In [ ]:
# a_sent = final_df['text'][0]
# test_sent

In [ ]:
# import time

In [ ]:
split1['text'][0]

"সমকামীতা'র হার বেড়েছে চট্টগ্রামে। নানা কারণে বিখ্যাত বন্দর নগরী চট্টগ্রাম। পাশ্চাত্যের অনেক ছোঁয়াই তাদের মাঝে বিরাজমান। সংস্কৃতির শেকড় বলা হয় চট্টগ্রামকে। বাংলার বুকে আরেক খন্ড বাংলা এই চট্টগ্রাম। তবে এত গুণাবলীর ছোঁয়া পেয়েও বাদ যায়নি চাটগাইয়াদের খারাপ অভ্যেসগুলোও।সম্প্রতি সমকামীতার মত কু-অভ্যাসটিও মাথা চাড়া দিয়ে উঠেছে এ অঞ্চলে। বিষয়টি নিছক গুজব বলে অনেকে উড়িয়ে দিলেও দায় এড়াতে পারেননি সারকাজম নিউজ টিম। সারকাজম নিউজের অনুসন্ধানী টিম বের করে নিয়ে আসে হাড়ির খবর।তিনদিন ব্যাপী চট্টগ্রামের বিভিন্ন এলাকা ঘুরে ঘটনার সত্যতার প্রমাণ মেলে। তবে এ যাত্রায় অল্পের জন্য পাছামারা খাননি সারকাজম নিউজ সাংবাদিক। চট্টগ্রাম বিশ্ববিদ্যালয়ের চারুকলা অনুষদের ছাত্র করিম চাকলাদার তার নাম প্রকাশ না করার শর্তে সারকাজম নিউজকে জানান, কয়েকদিন যাবত চট্টগ্রামে হালকা ঝিরিঝিরি বৃষ্টির কারণে সমকামীতা’র হার বৃদ্ধি পেয়েছে। পাছায় ব্লাকটেপ লাগিয়ে ঘুমাতে হচ্ছে।অনেকে নাকি পাছা পাহারা দেয়ার জন্য দারোয়ানও নিযুক্ত করেছেন। ধারণা করা হচ্ছে খুব শীঘ্রই ভারতের মত চিটাগাংয়েও সমকামীতা বৈধ হয়ে যাবে। – Sarcasm News Fun"

In [ ]:
split1['text'][1]

'৭ রোহিঙ্গা ধরে পুলিশে দিলো জনতা। সাতক্ষীরা সদর উপজেলার আবাদেরহাট এলাকা থেকে সাত রোহিঙ্গা নারী ও শিশুকে আটক করে পুলিশে দিয়েছেন স্থানীয়রা। বুধবার ভোরে তাদেরকে আটক করা হয়। আটকরা হলেন, মিয়ানমারের কুতুপালং ক্যাম্পের মো. সেলিমের স্ত্রী নুর বেগম (৪৫), তার মেয়ে রশিদা বেগম (৫), জান্নাতরা বেগম (৩), ছেলে মো. সাগর (১), আজিজুল হকের স্ত্রী হাছিনা বেগম (২২) ও তার মেয়ে রোজিনা বেগম (৪) ও উসুন জামান (৭৫)। আটক নুর বেগম জানান, ভারতে যাওয়ার জন্য দুই দালালের মাধ্যমে সাতক্ষীরা সীমান্তে আসেন তারা। দালালরা তাদেরকে কোলকাতায় ভালো কাজের প্রলোভন দেখিয়ে নিয়ে আসে। তারা আবাদের হাট এলাকায় পৌঁছালে স্থানীয় লোকজন তাদেরকে আটক করে পুলিশে দেয়। সাতক্ষীরা সদর থানার ভারপ্রাপ্ত কর্মকর্তা (ওসি) মোস্তাফিজুর রহমান জানান, সাত রোহিঙ্গা নারী ও শিশুকে ভারতে পাচারের উদ্দেশ্য নিয়ে আসার সময় স্থানীয় জনতা আটক করে পুলিশে দেয়। আকরামুল ইসলাম/এফএ/আরআইপি'

###batchwise summarize

In [ ]:
print('all file will be saved in: ',path2)

all file will be saved in:  /content/drive/MyDrive/datasets/sum_aug_train_augx2_cbuet_para.csv


In [ ]:
start=0
end = len(split1)
sum_count=0

# For every sentence...
for itr in range(start,end):
  sent = split1['text'][itr]   
  
  if(itr%10==0):
    split1.to_csv(path2, index=False) ##saving
    print(itr)
  
  if(calc_len_wtokenizer(sent)<512):
      continue        
    
  chunks,n = making_chunks(sent)

  for i in range(n):
    chunks[i] = mT5_summarize(chunks[i],n)
  
  sent = ' '.join(chunks)

  split1.at[itr,'text'] = sent
  sum_count+=1
  
print("successfully done. sum count: ", sum_count)

Token indices sequence length is longer than the specified maximum sequence length for this model (772 > 512). Running this sequence through the model will result in indexing errors


6190
6200
6210
6220
6230
6240
6250
6260
6270
6280
6290
6300
6310
6320
6330
6340
6350
6360
6370
6380
6390
6400
6410
6420
6430
6440
6450
6460
6470
6480
6490
6500
6510
6520
6530
6540
6550
6560
6570
6580
6590
6600
6610
6620
6630
6640
6650
6660
6670
6680
6690
6700
6710
6720
6730
6740
6750
6760
6770
6780
6790
6800
6810
6820
6830
6840
6850
6860
6870
6880
6890
6900
6910
6920
6930
6940
6950
6960
6970
6980
6990
7000
7010
successfully done. sum count:  416


In [ ]:
sum_count=0
max_len=0
for sent in split1['text']:   
  leng = calc_len_wtokenizer(sent)
  if(leng>max_len):
    max_len=leng
  if(leng>512):
    sum_count+=1

print(sum_count,max_len)

1 522


In [ ]:
split1.to_csv(path2, index=False)
print('done all. saved in:', path2)

done all. saved in: /content/drive/MyDrive/datasets/sum_aug_train_augx2_cbuet_para.csv
